In [7]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_curve, auc, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

Мы готовим данные чтобы тут было 50 на 50 протестить угадывает ли модель еденицы

In [8]:
valid_df = pd.read_csv("val.csv")

In [9]:
df_ones = valid_df[valid_df['target'] == 1]

df_zeros = valid_df[valid_df['target'] == 0].head(66)

filtered_df = pd.concat([df_zeros, df_ones])

filtered_df = filtered_df.reset_index(drop=True)
print(filtered_df)

     report_date  client_id  target  col1453  col1454  col1455  col1456  \
0              1          2       0        0       16      0.0      0.0   
1              2          4       0        0       18      0.0      0.0   
2              2          9       0        0       93      0.0      0.0   
3              0         12       0        0      108      0.0      0.0   
4              0         14       0        0      120      0.0      0.0   
..           ...        ...     ...      ...      ...      ...      ...   
127            0       2228       1        0     2160      0.0      0.0   
128            0       3256       1        0      773      0.0      0.0   
129            0       1150       1        0     2223      0.0      0.0   
130            1       1150       1        0     2223      0.0      1.0   
131            2       1974       1        0     2916      0.0      0.0   

     col1457  col1458  col1459  ...  col2588  col2589  col2590     col2654  \
0        0.0      0.0

In [10]:
filtered_df.to_csv('50.csv', index=False, encoding='utf-8')